In [1]:
import numpy as np
import pandas as pd
import time
import scripts.local_environment as local

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
%%time
df = pd.read_csv('data/clean/train_clean.csv')
df_targets = pd.read_csv('data/clean/train_labels.csv')
df_test = pd.read_csv('data/clean/test_clean.csv')

CPU times: user 49.7 s, sys: 4.6 s, total: 54.3 s
Wall time: 54.2 s


### Experimento 3
Entrenamiento con el mes de Mayo/2016 con sólo clientes antiguos

In [4]:
x_train = df[df['fecha_dato'] == '2016-05-28']
y_train = df_targets.loc[x_train.index]

In [5]:
ncodpers_both_month = list(set(x_train['ncodpers'].values) & set(df_test['ncodpers'].values))
len(ncodpers_both_month)

699690

In [6]:
x = x_train[x_train['ncodpers'].isin(ncodpers_both_month)].drop(['fecha_dato', 'fecha_alta'], axis=1)
y = y_train.loc[x.index].as_matrix()
x = x.as_matrix()

In [7]:
%%time
rf = local.model(x, y, RandomForestClassifier(n_jobs=4))

CPU times: user 2min 46s, sys: 8.94 s, total: 2min 55s
Wall time: 52.6 s


In [8]:
x_test = df_test.drop(['fecha_dato', 'fecha_alta'], axis=1).as_matrix()

In [9]:
%%time
probs, preds = local.calculatePredsProbs(x_test, rf)

CPU times: user 2min 16s, sys: 5.01 s, total: 2min 21s
Wall time: 1min 4s


In [15]:
%%time
subm = local.processPredictions(probs=probs, preds=preds, df_train=x_train, df_test=df_test,
                                df_targets=y_train, env='submit')

CPU times: user 17.1 s, sys: 828 ms, total: 18 s
Wall time: 19.1 s


### Experimento 4

In [4]:
x_train = df[df['fecha_dato'] == '2016-05-28']
y_train = df_targets.loc[x_train.index]

x = x_train.drop(['fecha_dato', 'fecha_alta'], axis=1)
y = y_train.loc[x.index].as_matrix()
x = x.as_matrix()

In [5]:
%%time
rf = local.model(x, y, RandomForestClassifier(n_jobs=4))

CPU times: user 2min 50s, sys: 8.83 s, total: 2min 59s
Wall time: 53.2 s


In [6]:
x_test = df_test.drop(['fecha_dato', 'fecha_alta'], axis=1).as_matrix()

In [7]:
%%time
probs, preds = local.calculatePredsProbs(x_test, rf)

CPU times: user 2min 16s, sys: 4.4 s, total: 2min 21s
Wall time: 1min 4s


In [9]:
%%time
subm, name_file = local.processPredictions(probs=probs, preds=preds, df_train=x_train, df_test=df_test,
                                df_targets=y_train, env='submit')

results/submissions/2018-07-13-h01-54-08_submission.csv
CPU times: user 17.2 s, sys: 788 ms, total: 18 s
Wall time: 19.1 s
